In [ ]:
import requests
import os

API_KEY = os.getenv("API_KEY")

SPORT = 'soccer_epl' 
REGIONS = 'uk' 
MARKETS = 'h2h' 
ODDS_FORMAT = 'decimal' 
DATE_FORMAT = 'iso' 
BETTING_EXCHANGES = set(["Matchbook", "Smarkets", "Betfair"])

SOCCER_KEYS = [
    "soccer_epl","soccer_efl_champ","soccer_england_league1","soccer_england_league2","soccer_england_efl_cup",
    "soccer_spl","soccer_league_of_ireland",
    "soccer_spain_la_liga","soccer_spain_segunda_division",
    "soccer_france_ligue_one","soccer_france_ligue_two",
    "soccer_germany_bundesliga","soccer_germany_bundesliga2","soccer_germany_liga3",
    "soccer_italy_serie_a","soccer_italy_serie_b",
    "soccer_portugal_primeira_liga","soccer_netherlands_eredivisie","soccer_belgium_first_div",
    "soccer_turkey_super_league","soccer_greece_super_league","soccer_switzerland_superleague",
    "soccer_austria_bundesliga","soccer_denmark_superliga","soccer_norway_eliteserien",
    "soccer_sweden_allsvenskan","soccer_sweden_superettan","soccer_finland_veikkausliiga","soccer_poland_ekstraklasa",
    "soccer_argentina_primera_division","soccer_brazil_campeonato","soccer_brazil_serie_b",
    "soccer_chile_campeonato","soccer_china_superleague","soccer_japan_j_league","soccer_korea_kleague1",
    "soccer_mexico_ligamx","soccer_usa_mls",
    "soccer_uefa_champs_league","soccer_uefa_europa_league",
    "soccer_conmebol_copa_libertadores","soccer_conmebol_copa_sudamericana",
    "soccer_australia_aleague",
]


In [3]:
odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()

    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])


Remaining requests 352
Used requests 148


In [4]:
if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()

    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])


Remaining requests 352
Used requests 148


In [5]:
import pandas as pd
from collections import defaultdict

leagues = []
for SPORT in SOCCER_KEYS:
    odds_response = requests.get(
        f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
        params={
            'api_key': API_KEY,
            'regions': REGIONS,
            'markets': MARKETS,
            'oddsFormat': ODDS_FORMAT,
            'dateFormat': DATE_FORMAT,
            'sport': SPORT,
            "commenceTimeTo": "2025-12-31T23:59:59Z"

        }
    )
    
    if odds_response.status_code != 200:
        print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

    else:
        odds_json = odds_response.json()

        print('Remaining requests', odds_response.headers['x-requests-remaining'])
        print('Used requests', odds_response.headers['x-requests-used'])
    df = pd.json_normalize(
        odds_json,
        record_path=['bookmakers','markets','outcomes'],
        meta=['id','sport_key','sport_title','commence_time','home_team','away_team',
                ['bookmakers','title'], ['bookmakers','last_update'],
                ['bookmakers','markets','last_update']],
        errors='ignore'
    )

    leagues.append(df)

parent = pd.concat(leagues)



Remaining requests 351
Used requests 149
Remaining requests 350
Used requests 150
Remaining requests 349
Used requests 151
Remaining requests 348
Used requests 152
Remaining requests 347
Used requests 153
Remaining requests 346
Used requests 154
Remaining requests 345
Used requests 155
Remaining requests 344
Used requests 156
Remaining requests 343
Used requests 157
Remaining requests 342
Used requests 158
Remaining requests 341
Used requests 159
Remaining requests 340
Used requests 160
Remaining requests 339
Used requests 161
Remaining requests 338
Used requests 162
Remaining requests 337
Used requests 163
Remaining requests 336
Used requests 164
Remaining requests 335
Used requests 165
Remaining requests 334
Used requests 166
Remaining requests 333
Used requests 167
Remaining requests 332
Used requests 168
Remaining requests 331
Used requests 169
Remaining requests 330
Used requests 170
Remaining requests 329
Used requests 171
Remaining requests 328
Used requests 172
Remaining reques

In [6]:
parent = (parent.rename(columns={
        'id':'event_id',
        'bookmakers.title':'bookmaker',
        'bookmakers.last_update':'bookmaker_last_update',
        'bookmakers.markets.last_update':'market_last_update',
        'name':'outcome_name',
        'price':'odds_decimal'
    })
    [['event_id','commence_time','home_team','away_team',
        'bookmaker','bookmaker_last_update',
        'outcome_name','odds_decimal']]
)

print(parent.tail(10))


                             event_id         commence_time  \
128  2c5c57eed06cfa8489b86070631b59e3  2025-11-01T06:00:00Z   
129  2c5c57eed06cfa8489b86070631b59e3  2025-11-01T06:00:00Z   
130  2c5c57eed06cfa8489b86070631b59e3  2025-11-01T06:00:00Z   
131  2c5c57eed06cfa8489b86070631b59e3  2025-11-01T06:00:00Z   
132  2c5c57eed06cfa8489b86070631b59e3  2025-11-01T06:00:00Z   
133  2c5c57eed06cfa8489b86070631b59e3  2025-11-01T06:00:00Z   
134  2c5c57eed06cfa8489b86070631b59e3  2025-11-01T06:00:00Z   
135  2c5c57eed06cfa8489b86070631b59e3  2025-11-01T06:00:00Z   
136  2c5c57eed06cfa8489b86070631b59e3  2025-11-01T06:00:00Z   
137  2c5c57eed06cfa8489b86070631b59e3  2025-11-01T06:00:00Z   

             home_team  away_team           bookmaker bookmaker_last_update  \
128  Newcastle Jets FC  Sydney FC            Smarkets  2025-10-25T21:25:31Z   
129  Newcastle Jets FC  Sydney FC            Smarkets  2025-10-25T21:25:31Z   
130  Newcastle Jets FC  Sydney FC            Smarkets  2025-10-25T21:

In [7]:
#{(home_team, away_team): {outcome: [(odds, bookmaker), ...]}}
odds_by_match = {}

for _, row in parent.iterrows():
    match_key = (row['home_team'], row['away_team'])
    outcome = row['outcome_name']
    odds = row['odds_decimal']
    bookmaker = row['bookmaker']
    
    if match_key not in odds_by_match:
        odds_by_match[match_key] = {}
    
    if outcome not in odds_by_match[match_key]:
        odds_by_match[match_key][outcome] = []
    
    odds_by_match[match_key][outcome].append((odds, bookmaker))


In [23]:
import pprint

BETTING_EXCHANGES = set(["Matchbook", "Smarkets", "Betfair"])

# accounts for slippage, ect
epsilon = 0.005

def calc_arbitrage(odds1, odds2, odds3):
    return (1/(odds1)) + (1/(odds2)) + (1/(odds3))

def find_arbritrage(odds_by_match):
    oppourtunities = []
    for match_key, outcomes in odds_by_match.items():

        if len(outcomes) != 3:
            continue

        best_h_val = 1 - epsilon
        current_best = None
    
        outcome1, outcome2, outcome3 = outcomes.keys()

        for odds1, bookie1 in outcomes[outcome1]:
            for odds2, bookie2 in outcomes[outcome2]:
                for odds3, bookie3 in outcomes[outcome3]:

                    if len({bookie1, bookie2, bookie3}) >= 2:
                        h_val = calc_arbitrage(odds1, odds2, odds3)
                        if h_val < best_h_val and not  bool({bookie1, bookie2, bookie3} & BETTING_EXCHANGES):
                            best_h_val = h_val
                            current_best = {
                                outcome1: (odds1, bookie1),
                                outcome2: (odds2, bookie2),
                                outcome3: (odds3, bookie3)
                            }

        if current_best:
            profit_margin = ((1/best_h_val)-1)*100
            contains_betting_exchange = bool({bookie1, bookie2, bookie3} & BETTING_EXCHANGES)
            oppourtunities.append({
                'match': f"{match_key[0]} vs {match_key[1]}",
                'h_val': best_h_val,
                'profit_margin_%': profit_margin,
                'combination': current_best,
                'contains_betting_exchange': contains_betting_exchange
            })
    
    return sorted(oppourtunities, key=lambda x: (x['contains_betting_exchange'], -x['h_val']))
    
arb_opps = find_arbritrage(odds_by_match)
pprint.pprint(arb_opps)


[{'combination': {'CF Estrela': (501.0, 'Casumo'),
                  'Draw': (17.0, 'Casumo'),
                  'Rio Ave FC': (1.1, 'Betway')},
  'contains_betting_exchange': False,
  'h_val': 0.9699104464867058,
  'match': 'CF Estrela vs Rio Ave FC',
  'profit_margin_%': 3.1023022406127643}]
